In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
import numpy as np
import pandas as pd
%matplotlib inline


In [3]:
from string import punctuation
from collections import Counter

In [4]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_xlim(0, 5)
ax.set_xlabel('Lunch')
ax.set_ylim(0, 5)
ax.set_ylabel('Viagra')


Text(0, 0.5, 'Viagra')

In [10]:
dir(ax)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_text',
 '_adjustable',
 '_agg_filter',
 '_alpha',
 '_anchor',
 '_animated',
 '_aspect',
 '_autoscaleXon',
 '_autoscaleYon',
 '_autotitlepos',
 '_axes',
 '_axes_class',
 '_axes_locator',
 '_axisbelow',
 '_cachedRenderer',
 '_clipon',
 '_clippath',
 '_connected',
 '_contains',
 '_current_image',
 '_facecolor',
 '_frameon',
 '_gci',
 '_gen_axes_patch',
 '_gen_axes_spines',
 '_get_axis_list',
 '_get_lines',
 '_get_patches_for_fill',
 '_get_view',
 '_gid',
 '_gridOn',
 '_hold',
 '_in_layout',
 '_init_axis',
 '_label',
 '_layoutbox',
 '_left_title',
 '_make_twin_axes',
 '_mouseover',
 '_m

In [14]:
documents = [
    """From: Martha Washington <marthafirst@hotmail.com>
Subject: Lunch
Date: January 25, 1794 9:30:58 AM EST
To: George Washington <president@whitehouse.gov>

Looking forward to lunch at noon today. I reserved my MealPal at Nerds and Nibblers. Please don't forget to book yours. You know how much I hate standing in line.

 xoxo Martha ðŸ˜»
""",
    """From: Aaron Burr <aabear@aol.com>
Subject: Quality meds, low price, quick delivery!
Date: January 25, 1794 9:35:21 AM EST
To: George Washington <president@whitehouse.gov>

Not dead yet? We have the Viagra that you need!!!

GUARANTEED BEST PRICE VIAGRA! WE HAVE QUICKSILVER TOO!!!

CLICK HERE FOR VIAGRA! ORDER NOW!
http://legit-pharmacy.ru/viagra-deals?ref=aabear

""",
    
]

In [18]:
vocab = ["lunch", "viagra"]

Steps to Vectorize Text (brainstormed in class)
Convert the string to lowercase
Split the string into words
Initialize something to store word counts
Tell it what words to look for
Count those words in each email

In [15]:
def replace_punctuation_with_spaces(text):
    """Return text with all punctuation turned into spaces."""
    output = ''
    for char in text:
        if char in punctuation:
            char = ' '
        output += char
    return output

In [16]:

[replace_punctuation_with_spaces(doc) for doc in documents]

['From  Martha Washington  marthafirst hotmail com \nSubject  Lunch\nDate  January 25  1794 9 30 58 AM EST\nTo  George Washington  president whitehouse gov \n\nLooking forward to lunch at noon today  I reserved my MealPal at Nerds and Nibblers  Please don t forget to book yours  You know how much I hate standing in line \n\n xoxo Martha ðŸ˜»\n',
 'From  Aaron Burr  aabear aol com \nSubject  Quality meds  low price  quick delivery \nDate  January 25  1794 9 35 21 AM EST\nTo  George Washington  president whitehouse gov \n\nNot dead yet  We have the Viagra that you need   \n\nGUARANTEED BEST PRICE VIAGRA  WE HAVE QUICKSILVER TOO   \n\nCLICK HERE FOR VIAGRA  ORDER NOW \nhttp   legit pharmacy ru viagra deals ref aabear\n\n']

In [19]:
def tokenize(doc, vocab=vocab):
    """Return a list of words in text."""
    text = replace_punctuation_with_spaces(doc.lower())
    words = [word for word in text.split() if word in vocab]
    return words

In [20]:
corpus = [tokenize(doc) for doc in documents]

In [21]:
def vectorize(corpus):
    """Return a DataFrame of the corpus."""
    return pd.DataFrame([Counter(doc) for doc in corpus]).fillna(0)

In [22]:
vectorize(corpus)

lunch  viagra
0    2.0     0.0
1    0.0     4.0

In [24]:
cv = CountVectorizer()  # Optionally: vocabulary=['lunch', 'viagra']
cv.fit(documents) # cv.vocabulary_  # e.g. {'from': 26, 'martha': 44, ...}
data = cv.transform(documents).toarray()

In [25]:
cols = [x[0] for x in sorted(cv.vocabulary_.items(), key=lambda x: x[0])]
data = pd.DataFrame(data, columns=cols)

In [26]:
data

1794  21  25  30  35  58  aabear  aaron  am  and  ...    today  too  \
0     1   0   1   1   0   1       0      0   1    1  ...        1    0   
1     1   1   1   0   1   0       2      1   1    0  ...        0    1   

   viagra  washington  we  whitehouse  xoxo  yet  you  yours  
0       0           2   0           1     1    0    1      1  
1       4           1   2           1     0    1    1      0  

[2 rows x 82 columns]

In [27]:
data.loc[:, ['lunch', 'viagra']]

lunch  viagra
0      2       0
1      0       4

My method: try making vectors out of emails with (number of "lunch", number of "viagra")

In [30]:
documents[0].count(vocab[0])

1

In [33]:
email0_vector = (documents[0].count(vocab[0]), documents[0].count(vocab[1]))
email0_vector

(1, 0)

In [35]:
email1_vector = (documents[1].count(vocab[0]), documents[1].count(vocab[1]))
email1_vector

(0, 1)

In [36]:
# count0 = (text0.count(vocab[0]), text0.count(vocab[1]))
# count0

NameError: name 'text0' is not defined

## Inverse document frequency allows us to shrink the magnitude of words that are incredibly common. A givens word's document frequency is the number of documents that feature that word. Then, take the number of documents and divide it by the document frequency. To scale it take the log.